### Konstantina Andronikou

## Transitive Vs Intransitive Verbs 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Intransitive 

In [4]:
# remove warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Carla pays")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'pays',
   'description': '[ARG1: Carla] [V: pays]',
   'tags': ['B-ARG1', 'B-V']}],
 'words': ['Carla', 'pays']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Carla pays"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'pays',
     'description': '[ARG1: Carla] [V: pays]',
     'tags': ['B-ARG1', 'B-V']}],
   'words': ['Carla', 'pays']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'first_name': 'Rose'}, {'first_name': 'Ron'}, {'first_name': 'Kathy'}, {'first_name': 'Deborah'}, {'first_name': 'Cynthia'}, {'first_name': 'Don'}, {'first_name': 'Susan'}, {'first_name': 'Mary'}, {'first_name': 'William'}, {'first_name': 'Sally'}, {'first_name': 'Ryan'}, {'first_name': 'Ben'}, {'first_name': 'Kelly'}, {'first_name': 'Kathleen'}, {'first_name': 'Rachel'}, {'first_name': 'Sarah'}, {'first_name': 'Rose'}, {'first_name': 'Laura'}, {'first_name': 'Edith'}, {'first_name': 'Kathy'}, {'first_name': 'Helen'}, {'first_name': 'Carl'}, {'first_name': 'Patrick'}, {'first_name': 'James'}, {'first_name': 'Rachel'}, {'first_name': 'Peter'}, {'first_name': 'Dorothy'}, {'first_name': 'Florence'}, {'first_name': 'Martin'}, {'first_name': 'Keith'}, {'first_name': 'Ryan'}, {'first_name': 'Mark'}, {'first_name': 'Frank'}, {'first_name': 'Sara'}, {'first_name': 'Rose'}, {'first_name': 'Angela'}, {'first_name': 'Robert'}, {'first_name': 'Johnny'}, {'first_name': 'Leslie'}, {'first_nam

In [12]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Intransitive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    312 (31.2%)

Example fails:
[ARG1: Kathy] [V: pays]
----
[ARG1: Rose] [V: pays]
----
[ARG1: Angela] [V: pays]
----
['Rose', 'pays'] [False]
['Ron', 'pays'] [ True]
['Kathy', 'pays'] [False]
['Deborah', 'pays'] [ True]
['Cynthia', 'pays'] [False]
['Don', 'pays'] [ True]
['Susan', 'pays'] [False]
['Mary', 'pays'] [ True]
['William', 'pays'] [ True]
['Sally', 'pays'] [False]
['Ryan', 'pays'] [ True]
['Ben', 'pays'] [ True]
['Kelly', 'pays'] [False]
['Kathleen', 'pays'] [False]
['Rachel', 'pays'] [ True]
['Sarah', 'pays'] [False]
['Rose', 'pays'] [False]
['Laura', 'pays'] [ True]
['Edith', 'pays'] [ True]
['Kathy', 'pays'] [False]
['Helen', 'pays'] [ True]
['Carl', 'pays'] [ True]
['Patrick', 'pays'] [False]
['James', 'pays'] [ True]
['Rachel', 'pays'] [ True]
['Peter', 'pays'] [ True]
['Dorothy', 'pays'] [False]
['Florence', 'pays'] [False]
['Martin', 'pays'] [ True]
['Keith', 'pays'] [ True]
['Ryan', 'pays'] [ True]
['Mark',

In [13]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Intransitive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive 

In [14]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [15]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [16]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [17]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [18]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Jane'}, {'first_name': 'Jennifer'}, {'first_name': 'Heather'}, {'first_name': 'Nancy'}, {'first_name': 'Helen'}, {'first_name': 'Ed'}, {'first_name': 'Philip'}, {'first_name': 'Sandra'}, {'first_name': 'Jill'}, {'first_name': 'David'}, {'first_name': 'Nicole'}, {'first_name': 'Sandra'}, {'first_name': 'Matt'}, {'first_name': 'Alexander'}, {'first_name': 'Sarah'}, {'first_name': 'Colin'}, {'first_name': 'Kevin'}, {'first_name': 'Andrew'}, {'first_name': 'Emma'}, {'first_name': 'Ann'}, {'first_name': 'Harry'}, {'first_name': 'Larry'}, {'first_name': 'Jim'}, {'first_name': 'Alice'}, {'first_name': 'Carolyn'}, {'first_name': 'Emma'}, {'first_name': 'Lynn'}, {'first_name': 'Sally'}, {'first_name': 'Keith'}, {'first_name': 'John'}, {'first_name': 'Deborah'}, {'first_name': 'Robin'}, {'first_name': 'Robin'}, {'first_name': 'Heather'}, {'first_name': 'Alex'}, {'first_name': 'Anne'}, {'first_name': 'Sandra'}, {'first_name': 'Christopher'}, {'first_name': 'James'}, {'first_

In [19]:
with open('dataset/Transitive.txt', 'w') as f:
    print(t.data, file = f)

In [20]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Transitive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Jane', 'pays', 'for', 'coffee', '.'] [ True]
['Jennifer', 'pays', 'for', 'coffee', '.'] [ True]
['Heather', 'pays', 'for', 'coffee', '.'] [ True]
['Nancy', 'pays', 'for', 'coffee', '.'] [ True]
['Helen', 'pays', 'for', 'coffee', '.'] [ True]
['Ed', 'pays', 'for', 'coffee', '.'] [ True]
['Philip', 'pays', 'for', 'coffee', '.'] [ True]
['Sandra', 'pays', 'for', 'coffee', '.'] [ True]
['Jill', 'pays', 'for', 'coffee', '.'] [ True]
['David', 'pays', 'for', 'coffee', '.'] [ True]
['Nicole', 'pays', 'for', 'coffee', '.'] [ True]
['Sandra', 'pays', 'for', 'coffee', '.'] [ True]
['Matt', 'pays', 'for', 'coffee', '.'] [ True]
['Alexander', 'pays', 'for', 'coffee', '.'] [ True]
['Sarah', 'pays', 'for', 'coffee', '.'] [ True]
['Colin', 'pays', 'for', 'coffee', '.'] [ True]
['Kevin', 'pays', 'for', 'coffee', '.'] [ True]
['Andrew', 'pays', 'for', 'coffee', '.'] [ True]
['Emma', 'pays', 'for', 'coffee', '.'] [ True]
['Ann', 

In [21]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Transitive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive-Intransitive Robustness Test

In [22]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Laura'}, {'first_name': 'Sue'}, {'first_name': 'Bill'}, {'first_name': 'Janet'}, {'first_name': 'Ian'}, {'first_name': 'Fiona'}, {'first_name': 'Billy'}, {'first_name': 'Jeff'}, {'first_name': 'Amy'}, {'first_name': 'Mark'}, {'first_name': 'Jonathan'}, {'first_name': 'Carolyn'}, {'first_name': 'Jerry'}, {'first_name': 'Steve'}, {'first_name': 'Kenneth'}, {'first_name': 'Anne'}, {'first_name': 'Roy'}, {'first_name': 'Elizabeth'}, {'first_name': 'Kenneth'}, {'first_name': 'Lucy'}, {'first_name': 'Matthew'}, {'first_name': 'Leslie'}, {'first_name': 'Alexander'}, {'first_name': 'Ruth'}, {'first_name': 'Alan'}, {'first_name': 'Roy'}, {'first_name': 'Ann'}, {'first_name': 'Paul'}, {'first_name': 'Ian'}, {'first_name': 'Ruth'}, {'first_name': 'Larry'}, {'first_name': 'David'}, {'first_name': 'Michael'}, {'first_name': 'Linda'}, {'first_name': 'Nicole'}, {'first_name': 'Jim'}, {'first_name': 'Thomas'}, {'first_name': 'Gary'}, {'first_name': 'Chris'}, {'first_name': 'Jonat

In [23]:
with open('dataset/Intransitive_Rob_BERT.txt', 'w') as f:
    print(t.data, file = f)

In [24]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Intransitive_Rob_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    22 (2.2%)

Example fails:
[ARG1: Rose] [V: payed] .
----
[ARG1: Bill] [V: payed] .
----
[ARG1: Rose] [V: payed] .
----
['Laura', 'payed', '.'] [ True]
['Sue', 'payed', '.'] [ True]
['Bill', 'payed', '.'] [False]
['Janet', 'payed', '.'] [ True]
['Ian', 'payed', '.'] [ True]
['Fiona', 'payed', '.'] [ True]
['Billy', 'payed', '.'] [ True]
['Jeff', 'payed', '.'] [ True]
['Amy', 'payed', '.'] [ True]
['Mark', 'payed', '.'] [ True]
['Jonathan', 'payed', '.'] [ True]
['Carolyn', 'payed', '.'] [ True]
['Jerry', 'payed', '.'] [ True]
['Steve', 'payed', '.'] [ True]
['Kenneth', 'payed', '.'] [ True]
['Anne', 'payed', '.'] [ True]
['Roy', 'payed', '.'] [ True]
['Elizabeth', 'payed', '.'] [ True]
['Kenneth', 'payed', '.'] [ True]
['Lucy', 'payed', '.'] [ True]
['Matthew', 'payed', '.'] [ True]
['Leslie', 'payed', '.'] [ True]
['Alexander', 'payed', '.'] [ True]
['Ruth', 'payed', '.'] [ True]
['Alan', 'payed', '.'] [ True]
['Roy', 'pay

In [25]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Intransitive_Rob_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive

In [26]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Ian'}, {'first_name': 'William'}, {'first_name': 'Bob'}, {'first_name': 'Robin'}, {'first_name': 'Arthur'}, {'first_name': 'Sue'}, {'first_name': 'Harriet'}, {'first_name': 'Ryan'}, {'first_name': 'Jane'}, {'first_name': 'Tim'}, {'first_name': 'Ralph'}, {'first_name': 'Henry'}, {'first_name': 'Victoria'}, {'first_name': 'Susan'}, {'first_name': 'Marilyn'}, {'first_name': 'Jill'}, {'first_name': 'Judy'}, {'first_name': 'Martha'}, {'first_name': 'Matthew'}, {'first_name': 'Jeff'}, {'first_name': 'Heather'}, {'first_name': 'Dick'}, {'first_name': 'Eric'}, {'first_name': 'Ellen'}, {'first_name': 'Jennifer'}, {'first_name': 'Patricia'}, {'first_name': 'Benjamin'}, {'first_name': 'Christine'}, {'first_name': 'Frank'}, {'first_name': 'Fred'}, {'first_name': 'Margaret'}, {'first_name': 'Maria'}, {'first_name': 'Pamela'}, {'first_name': 'Anna'}, {'first_name': 'Charlotte'}, {'first_name': 'Alexandra'}, {'first_name': 'Leslie'}, {'first_name': 'Matthew'}, {'first_name': 'Ch

In [27]:
with open('dataset/Transitive_Rob_BERT.txt', 'w') as f:
    print(t.data, file = f)

In [28]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Transitive_Rob_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Ian', 'payed', 'for', 'coffee', '.'] [ True]
['William', 'payed', 'for', 'coffee', '.'] [ True]
['Bob', 'payed', 'for', 'coffee', '.'] [ True]
['Robin', 'payed', 'for', 'coffee', '.'] [ True]
['Arthur', 'payed', 'for', 'coffee', '.'] [ True]
['Sue', 'payed', 'for', 'coffee', '.'] [ True]
['Harriet', 'payed', 'for', 'coffee', '.'] [ True]
['Ryan', 'payed', 'for', 'coffee', '.'] [ True]
['Jane', 'payed', 'for', 'coffee', '.'] [ True]
['Tim', 'payed', 'for', 'coffee', '.'] [ True]
['Ralph', 'payed', 'for', 'coffee', '.'] [ True]
['Henry', 'payed', 'for', 'coffee', '.'] [ True]
['Victoria', 'payed', 'for', 'coffee', '.'] [ True]
['Susan', 'payed', 'for', 'coffee', '.'] [ True]
['Marilyn', 'payed', 'for', 'coffee', '.'] [ True]
['Jill', 'payed', 'for', 'coffee', '.'] [ True]
['Judy', 'payed', 'for', 'coffee', '.'] [ True]
['Martha', 'payed', 'for', 'coffee', '.'] [ True]
['Matthew', 'payed', 'for', 'coffee', '.'] [ T

In [29]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Transitive_Rob_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 